In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r ./drive/MyDrive/hackathon/sent-emb/requirements.txt

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 

In [ ]:
!pip install numpy==1.23.3

  Using cached numpy-1.23.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 49.3.0 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.3 which is incompatible.
cvxpy 1.3.4 requires setuptools>65.5.1, but you have setuptools 49.3.0 which is incompatible.
dopamine-rl 4.0.9 requires tqdm>=4.64.1, but you have tqdm 4.49.0 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.3 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.3 which is incom

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/hackathon/sent-emb/src/')

In [ ]:
from prettytable import PrettyTable
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM
from dataset import STS12Eval, STS13Eval, STS14Eval, STS15Eval, STS16Eval
import argparse
import torch
from huggingface_hub import login
from tqdm import tqdm
import numpy as np

In [ ]:
# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

# Set up util functions

In [ ]:
# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def print_table(task_names, scores):
    tb = PrettyTable()
    tb.field_names = task_names
    tb.add_row(scores)
    print(tb)


# argparsers

In [ ]:
# model_name_or_path = 'meta-llama/Meta-Llama-3-8B'
model_name_or_path = 'openai-community/gpt2'
# model_name_or_path = 'facebook/opt-125m'

In [ ]:
token = 'YOUR TOKEN HERE'
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load the LLM
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                device_map='auto',
                                                output_hidden_states=True,
                                                trust_remote_code=True,)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = 0  # Set the padding token.
tokenizer.padding_side = "left"  # Allow batched inference

# Device setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Batcher is the KEY function that you need to modify to develop you own embedding extraction method
####################################################################################################
def batcher(params, batch, max_length=512):
    # Handle rare token encoding issues in the dataset
    if len(batch) >= 1 and len(batch[0]) >= 1 and isinstance(batch[0][0], bytes):
        batch = [[word.decode('utf-8') for word in s] for s in batch]

    sentences = [' '.join(s) for s in batch]

    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors='pt',
        padding=True,
        max_length=max_length,
        truncation=max_length is not None
    )

    # Move to the correct device
    for k in batch:
        batch[k] = batch[k].to(device) if batch[k] is not None else None

    # Get raw embeddings
    with torch.no_grad():
        hidden_states = model(output_hidden_states=True, return_dict=True, **batch).hidden_states

        last_layer = hidden_states[-2]
        outputs = ((last_layer * batch['attention_mask'].unsqueeze(-1)).sum(1) / batch['attention_mask'].sum(-1).unsqueeze(-1))
        if outputs.dtype == torch.bfloat16:
            # bfloat16 not support for .numpy()
            outputs = outputs.float()

    return outputs.cpu()
####################################################################################################


In [ ]:
# Obtain the embeddings and evaluate them
results = {}
tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16']
PATH_TO_DATA = "./drive/MyDrive/hackathon/sent-emb/data/STS/"
params = dotdict({'task_path': PATH_TO_DATA, 'batch_size':16})
eval_dict = {"STS12": STS12Eval, "STS13": STS13Eval, "STS14": STS14Eval, "STS15": STS15Eval, "STS16": STS16Eval}
for task in tqdm(tasks):
    fpath = task + '-en-test'
    evaluation = eval_dict[task](params.task_path + fpath) # Load the dataset
    result = evaluation.run(params, batcher) # Obtain the embeddings and evaluate them
    results[task] = result

100%|██████████| 5/5 [36:47<00:00, 441.47s/it]


In [ ]:
# Print results using print_table()
task_names = []
scores = []
for task in ['STS12', 'STS13', 'STS14', 'STS15', 'STS16']:
    task_names.append(task)
    if task in results:
        scores.append("%.2f" % (results[task]['all']['spearman']['all'] * 100))
    else:
        scores.append("0.00")
task_names.append("Avg.")
scores.append("%.2f" % (sum([float(score) for score in scores]) / len(tasks)))
print_table(task_names, scores)

+-------+-------+-------+-------+-------+-------+
| STS12 | STS13 | STS14 | STS15 | STS16 |  Avg. |
+-------+-------+-------+-------+-------+-------+
|  6.13 | 20.04 | 11.85 | 28.30 | 21.20 | 17.50 |
+-------+-------+-------+-------+-------+-------+


In [ ]:
# Write results to file
with open('./drive/MyDrive/hackathon/sent-emb/sts-org-results.txt', 'a') as f:
    model_name = model_name_or_path.split('/')[-1]
    f.write(model_name + ' ' + ' '.join([str(s) for s in scores]) + ' '+'-2\n')
